<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code and text of the response:

In [3]:
#ANSWER - The code checks if the request was successful (HTTP status code 200 indicates success)
print(response.status_code)

200


In [4]:
#ANSWER - Text of the response
print(response.text)

{"timestamp": 1695716536, "message": "success", "iss_position": {"longitude": "50.4035", "latitude": "-8.2453"}}


We can use another API to request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21).

In [5]:
response = requests.get("https://api.g7vrd.co.uk/v1/satellite-passes/25544/-33.87/151.21.json?minelevation=0&hours=24")

Print the response header:

In [6]:
#ANSWER
print(response.headers)

{'Date': 'Tue, 26 Sep 2023 08:36:25 GMT', 'Server': 'Apache', 'Vary': 'Origin,Access-Control-Request-Method,Access-Control-Request-Headers', 'Access-Control-Allow-Origin': '*', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Content-Type': 'application/json', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}


Print the content of the response (the data that the server returned):

In [7]:
#ANSWER
print(response.content)

b'{"api_status":"ALPHA","request_timestamp":"2023-09-26T08:36:25.451149316Z","norad_id":25544,"satellite_name":"ISS","tle_last_retrieved":"2023-09-25T16:31:22.073123936Z","lat":-33.87,"lon":151.21,"hours":24,"min_elevation":0,"query_ms":26,"passes":[{"start":"2023-09-26T15:28:00.425Z","tca":"2023-09-26T15:33:00.425Z","end":"2023-09-26T15:38:40.425Z","aos_azimuth":332,"los_azimuth":125,"max_elevation":36.0},{"start":"2023-09-26T17:05:00.425Z","tca":"2023-09-26T17:10:00.425Z","end":"2023-09-26T17:15:20.425Z","aos_azimuth":281,"los_azimuth":144,"max_elevation":21.0},{"start":"2023-09-26T18:44:30.425Z","tca":"2023-09-26T18:48:00.425Z","end":"2023-09-26T18:51:35.425Z","aos_azimuth":234,"los_azimuth":153,"max_elevation":5.0},{"start":"2023-09-26T20:23:30.425Z","tca":"2023-09-26T20:26:30.425Z","end":"2023-09-26T20:29:40.425Z","aos_azimuth":206,"los_azimuth":138,"max_elevation":3.0},{"start":"2023-09-26T22:00:05.425Z","tca":"2023-09-26T22:04:35.425Z","end":"2023-09-26T22:09:25.425Z","aos_azimu

Note that this is a Python byte string:

In [8]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [9]:
#ANSWER
print(response.headers["content-type"])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [10]:
overheads = response.json()
print(overheads)

{'api_status': 'ALPHA', 'request_timestamp': '2023-09-26T08:36:25.451149316Z', 'norad_id': 25544, 'satellite_name': 'ISS', 'tle_last_retrieved': '2023-09-25T16:31:22.073123936Z', 'lat': -33.87, 'lon': 151.21, 'hours': 24, 'min_elevation': 0, 'query_ms': 26, 'passes': [{'start': '2023-09-26T15:28:00.425Z', 'tca': '2023-09-26T15:33:00.425Z', 'end': '2023-09-26T15:38:40.425Z', 'aos_azimuth': 332, 'los_azimuth': 125, 'max_elevation': 36.0}, {'start': '2023-09-26T17:05:00.425Z', 'tca': '2023-09-26T17:10:00.425Z', 'end': '2023-09-26T17:15:20.425Z', 'aos_azimuth': 281, 'los_azimuth': 144, 'max_elevation': 21.0}, {'start': '2023-09-26T18:44:30.425Z', 'tca': '2023-09-26T18:48:00.425Z', 'end': '2023-09-26T18:51:35.425Z', 'aos_azimuth': 234, 'los_azimuth': 153, 'max_elevation': 5.0}, {'start': '2023-09-26T20:23:30.425Z', 'tca': '2023-09-26T20:26:30.425Z', 'end': '2023-09-26T20:29:40.425Z', 'aos_azimuth': 206, 'los_azimuth': 138, 'max_elevation': 3.0}, {'start': '2023-09-26T22:00:05.425Z', 'tca': 

What kind of object did this give us?

In [11]:
#ANSWER:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `passes` value to a separate list:

In [12]:
#ANSWER:
passes = overheads['passes']

Now extract the `start` strings into an array called `srisetimes`:

In [13]:
#ANSWER: 'start' is presumably the start time of the satellite pass
srisetimes = [xpass['start'] for xpass in passes]
srisetimes

['2023-09-26T15:28:00.425Z',
 '2023-09-26T17:05:00.425Z',
 '2023-09-26T18:44:30.425Z',
 '2023-09-26T20:23:30.425Z',
 '2023-09-26T22:00:05.425Z',
 '2023-09-26T23:36:25.425Z',
 '2023-09-27T01:14:30.425Z',
 '2023-09-27T14:40:20.425Z']

These are strings. We convert these to an array of Python `datetime` values called `risetimes`:

In [14]:
risetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]
risetimes

[datetime.datetime(2023, 9, 26, 15, 28, 0, 425000),
 datetime.datetime(2023, 9, 26, 17, 5, 0, 425000),
 datetime.datetime(2023, 9, 26, 18, 44, 30, 425000),
 datetime.datetime(2023, 9, 26, 20, 23, 30, 425000),
 datetime.datetime(2023, 9, 26, 22, 0, 5, 425000),
 datetime.datetime(2023, 9, 26, 23, 36, 25, 425000),
 datetime.datetime(2023, 9, 27, 1, 14, 30, 425000),
 datetime.datetime(2023, 9, 27, 14, 40, 20, 425000)]

Finally, use `risetime.strftime` to print these in a format that people understand:

```
e.g.
18/10/22 07:05
18/10/22 08:41
18/10/22 10:20
18/10/22 12:00
18/10/22 01:37
18/10/22 03:13
```



In [16]:
#ANSWER:
for risetime in risetimes:
    print(risetime.strftime('%d/%m/%y %I:%M'))

26/09/23 03:28
26/09/23 05:05
26/09/23 06:44
26/09/23 08:23
26/09/23 10:00
26/09/23 11:36
27/09/23 01:14
27/09/23 02:40


Finally, here is an endpoint that tells us who is on board:

In [17]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [18]:
#ANSWER:
astros = response.json()
print(astros)

{'people': [{'name': 'Sergey Prokopyev', 'craft': 'ISS'}, {'name': 'Dmitry Petelin', 'craft': 'ISS'}, {'name': 'Frank Rubio', 'craft': 'ISS'}, {'name': 'Jing Haiping', 'craft': 'Tiangong'}, {'name': 'Gui Haichow', 'craft': 'Tiangong'}, {'name': 'Zhu Yangzhu', 'craft': 'Tiangong'}, {'name': 'Jasmin Moghbeli', 'craft': 'ISS'}, {'name': 'Andreas Mogensen', 'craft': 'ISS'}, {'name': 'Satoshi Furukawa', 'craft': 'ISS'}, {'name': 'Konstantin Borisov', 'craft': 'ISS'}, {'name': 'Oleg Kononenko', 'craft': 'ISS'}, {'name': 'Nikolai Chub', 'craft': 'ISS'}, {'name': "Loral O'Hara", 'craft': 'ISS'}], 'number': 13, 'message': 'success'}


In [19]:
print(astros['number'])

13


In [20]:
for astronaut in astros['people']:
    print(astronaut['name'])

Sergey Prokopyev
Dmitry Petelin
Frank Rubio
Jing Haiping
Gui Haichow
Zhu Yangzhu
Jasmin Moghbeli
Andreas Mogensen
Satoshi Furukawa
Konstantin Borisov
Oleg Kononenko
Nikolai Chub
Loral O'Hara


## HOMEWORK


1. Write a simple error handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the error handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [22]:
#ANSWER:
def handleResponse(response, verbose = False):
    if response.status_code == 200:
        if verbose:
            print('Response = 200: OK')
        fail = False
    else:
        if verbose:
            print('Response = ' + str(response.status_code) + ': ' + '(TODO: output of lookup function here)')
        fail = True
    return(fail, response.status_code)

2. Test your response handler on some correct and incorrect API calls.

In [23]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-now.json")
handleResponse(response, True)[0]

Response = 200: OK


False

>

>

>



---



---



> > > > > > > > > © 2022 Institute of Data


---



---



